In [1]:
import os
import sys
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset
import torchaudio
import soundfile
import matplotlib.pyplot as plt
import numpy as np
from pydub import AudioSegment

module_path = os.path.abspath(os.path.join('./source/'))
if module_path not in sys.path:
    sys.path.append(module_path)

from model import DiffWave
from dataset import ChunkedMusDBHQ
from train import train

c:\Users\hause\miniconda3\envs\practical_work\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
chunked_data = ChunkedMusDBHQ(audio_dir='data/chunked_audio/')

trainloader = torch.utils.data.DataLoader(
    chunked_data,
    batch_size=2,
    shuffle=True,
    )

In [3]:
variance_schedule = torch.linspace(0.001, 0.05, 50)
print(variance_schedule)

tensor([0.0010, 0.0020, 0.0030, 0.0040, 0.0050, 0.0060, 0.0070, 0.0080, 0.0090,
        0.0100, 0.0110, 0.0120, 0.0130, 0.0140, 0.0150, 0.0160, 0.0170, 0.0180,
        0.0190, 0.0200, 0.0210, 0.0220, 0.0230, 0.0240, 0.0250, 0.0260, 0.0270,
        0.0280, 0.0290, 0.0300, 0.0310, 0.0320, 0.0330, 0.0340, 0.0350, 0.0360,
        0.0370, 0.0380, 0.0390, 0.0400, 0.0410, 0.0420, 0.0430, 0.0440, 0.0450,
        0.0460, 0.0470, 0.0480, 0.0490, 0.0500])


In [4]:

model = train(8, 4, trainloader, 1, len(variance_schedule), variance_schedule) #C, num_blocks, trainloader, epochs, timesteps, variance_schedule

epoch: 0 | batch: 0 | loss: 0.4485573172569275
epoch: 0 | batch: 5 | loss: 0.7986760139465332
epoch: 0 | batch: 10 | loss: 0.26053935289382935
epoch: 0 | batch: 15 | loss: 0.18985752761363983
Epoch 0 DONE!


In [5]:
x_t = torch.randn(1, 1, 220500)
sample = model.sample(x_t)

### trying out stuff!

In [ ]:
#test forward pass with one sample
sample = next(iter(trainloader)) #waveform, sample_rate
waveform = sample[0]
print(waveform.shape) #batch size, num channels, sample length

#only take first channel of sample; indexing is done with 0:1 instead of [0], otherwise shape is not kept the same
waveform = waveform[:,0:1,:]

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DiffWave(32)
model = model.to(device)

In [ ]:
variance_schedule = torch.tensor([0.1,0.2,0.3,0.4,0.5])
test_model = DiffWave(64, 5, len(variance_schedule), variance_schedule)

In [ ]:
test_model